# HW6-DQN


In this exercise you are going to implement your first keras-rl agent based on the **Acrobot** environment (https://www.gymlibrary.dev/environments/classic_control/acrobot/) <br />
The goal of this environment is to maneuver the robot arm upwards above the line with as little steps as possible

**TASK: Import necessary libraries** <br />

In [1]:
!pip install tensorflow==2.8.0
!pip install keras-rl2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 497.6/497.6 MB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 78.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.5/462.5 kB 23.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 58.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 67.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 781.3/781.3 kB 44.3 MB/s eta 0:00:00
  Attempting uninstall: keras
    Found existing installation: keras 2.12.0
    Uninstalling keras-2.12.0:
      Successfully uninstalled keras-2.12.0
  Attempting uninstall: tensorboard-data-server
    Found existing installation: tensorboard-data-server 0.7.1
    Uninstalling tensorboard-data-server-0.7.1:
      Successfully uninstalled tensorboard-data-server-0.7.1
  Attempting uninstall: google-auth-oauthlib
    Found existing insta

In [2]:
##TODO
!pip install gym


**TASK: Create the environment** <br />
The name is: *Acrobot-v1*

In [3]:
##TODO
import gym
import numpy as np

# Create the Acrobot environment
env = gym.make('Acrobot-v1')


/usr/local/lib/python3.10/dist-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(


In [4]:
num_actions = env.action_space.n
num_observations = env.observation_space.shape
print(f"Action Space: {env.action_space.n}")
print(f"Observation Space: {num_observations}")

assert num_actions == 3 and num_observations == (6,) , "Wrong environment!"

Action Space: 3
Observation Space: (6,)


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


**TASK: Create the Neural Network for your Deep-Q-Agent** <br />
Take a look at the size of the action space and the size of the observation space.
You are free to chose any architecture you want! <br />
Hint: It already works with three layers, each having 64 neurons.

In [5]:
## TODO
!pip install torch


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [6]:
#Create neural networks
import torch
import torch.nn as nn
import torch.nn.functional as F

class QNetwork(nn.Module):
    def __init__(self, input_size, output_size):
        super(QNetwork, self).__init__()
        self.fc1 = nn.Linear(input_size, 64)
        self.fc2 = nn.Linear(64, 64)
        self.fc3 = nn.Linear(64, output_size)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Define input size based on observation space and output size based on action space
input_size = env.observation_space.shape[0]
output_size = env.action_space.n

# Create the Q-network
q_network = QNetwork(input_size, output_size)


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


**TASK: Initialize the circular buffer**<br />
Make sure you set the limit appropriately (50000 works well)

In [7]:
## TODO
from collections import deque

# Initialize the replay memory (circular buffer)
replay_memory_limit = 50000
replay_memory = deque(maxlen=replay_memory_limit)


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


**TASK: Use the epsilon greedy action selection strategy with *decaying* epsilon**

In [12]:
## TODO
# Hyperparameters
initial_epsilon = 1.0
min_epsilon = 0.01
epsilon_decay = 0.995

# Initialize epsilon
epsilon = initial_epsilon

# Epsilon-greedy action selection
def select_action(state):
    if np.random.rand() < epsilon:
        return env.action_space.sample()  # Explore
    else:
        with torch.no_grad():
            state_tensor = torch.FloatTensor(state)
            q_values = q_network(state_tensor)
            return q_values.argmax().item()  # Exploit

num_episodes = 1000  # Define the number of episodes
print_interval = 10  # Print status every 10 episodes

# During the training loop
for episode in range(num_episodes):
    state = env.reset()  # Reset the environment for a new episode
    done = False  # Initialize the "done" flag

    while not done:
        action = select_action(state)
        next_state, reward, done, _ = env.step(action)  # Get next state and reward
        # Update the Q-values, experience replay, etc.
        state = next_state

    # Decay epsilon
    epsilon = max(min_epsilon, epsilon * epsilon_decay)

    # Print status every print_interval episodes
    if (episode + 1) % print_interval == 0:
        print(f"Episode {episode+1}, Epsilon: {epsilon:.4f}")


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Episode 10, Epsilon: 0.9511
Episode 20, Epsilon: 0.9046
Episode 30, Epsilon: 0.8604
Episode 40, Epsilon: 0.8183
Episode 50, Epsilon: 0.7783
Episode 60, Epsilon: 0.7403
Episode 70, Epsilon: 0.7041
Episode 80, Epsilon: 0.6696
Episode 90, Epsilon: 0.6369
Episode 100, Epsilon: 0.6058
Episode 110, Epsilon: 0.5762
Episode 120, Epsilon: 0.5480
Episode 130, Epsilon: 0.5212
Episode 140, Epsilon: 0.4957
Episode 150, Epsilon: 0.4715
Episode 160, Epsilon: 0.4484
Episode 170, Epsilon: 0.4265
Episode 180, Epsilon: 0.4057
Episode 190, Epsilon: 0.3858
Episode 200, Epsilon: 0.3670
Episode 210, Epsilon: 0.3490
Episode 220, Epsilon: 0.3320
Episode 230, Epsilon: 0.3157
Episode 240, Epsilon: 0.3003
Episode 250, Epsilon: 0.2856
Episode 260, Epsilon: 0.2716
Episode 270, Epsilon: 0.2584
Episode 280, Epsilon: 0.2457
Episode 290, Epsilon: 0.2337
Episode 300, Epsilon: 0.2223
Episode 310, Epsilon: 0.2114
Episode 320, Epsilon: 0.2011
Episode 330, Epsilon: 0.1913
Episode 340, Epsilon: 0.1819
Episode 350, Epsilon: 0

**TASK: Create the DQNAgent** <br />
Feel free to play with the nb_steps_warump, target_model_update, batch_size and gamma parameters. <br />
Hint:<br />
You can try *nb_steps_warmup*=1000, *target_model_update*=1000, *batch_size*=32 and *gamma*=0.99 as a first guess

In [26]:
from rl.policy import EpsGreedyQPolicy
from rl.memory import SequentialMemory
from rl.agents.dqn import DQNAgent
from keras.models import Sequential
from keras.layers import Dense

# Create the Q-network model
input_size = 6
output_size = 3
q_network = Sequential([
    Dense(64, activation='relu', input_shape=(input_size,)),
    Dense(64, activation='relu'),
    Dense(output_size, activation='linear')  # Linear activation for Q-values
])

# Parameters for DQNAgent
nb_steps_warmup = 1000
target_model_update = 1000
batch_size = 32
gamma = 0.99

# Define policy and memory
policy = EpsGreedyQPolicy()
memory = SequentialMemory(limit=replay_memory_limit, window_length=1)

# Create the DQNAgent using the q_network model
dqn_agent = DQNAgent(model=q_network, nb_actions=output_size, policy=policy, memory=memory,
                     nb_steps_warmup=nb_steps_warmup, target_model_update=target_model_update,
                     batch_size=batch_size, gamma=gamma)


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


**TASK: Compile the model** <br />
Feel free to explore the effects of different optimizers and learning rates.
You can try Adam with a learning rate of 1e-3 as a first guess

In [27]:
## TODO
from tensorflow.keras.optimizers import Adam

# Define optimizer and learning rate
optimizer = Adam(learning_rate=1e-3)

# Compile the DQN model
dqn_agent.compile(optimizer=optimizer, metrics=['mae'])


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


**TASK: Fit the model** <br />
150,000 steps should be a very good starting point

In [ ]:
## TODO
from tensorflow.keras.optimizers import Adam

# Define optimizer and learning rate
optimizer = Adam(learning_rate=1e-3)

# Compile the DQN model
dqn_agent.compile(optimizer=optimizer, metrics=['mae'])


**TASK: Evaluate the model**

In [ ]:
## TODO
## TODO
# Train the DQN agent
history = dqn_agent.fit(env, nb_steps=150000, visualize=False, verbose=1)



In [ ]:
# Assuming you've already created the DQNAgent and defined the Q-network as shown earlier

# Import necessary libraries
import gym

# Create the environment
env = gym.make('Acrobot-v1')

# Define input size and output size
input_size = env.observation_space.shape[0]
output_size = env.action_space.n

# Train the DQN agent
history = dqn_agent.fit(env, nb_steps=150000, visualize=False, verbose=1)
